# Libraries

In [2]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
from dotenv import load_dotenv
from sqlalchemy import text
import pandas as pd
import psycopg2
import os

In [3]:
load_dotenv()

True

# Database

In [4]:
POSTGRES_USER = os.getenv('POSTGRES_USER')
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
POSTGRES_HOST = os.getenv('POSTGRES_HOST')
POSTGRES_PORT = os.getenv('POSTGRES_PORT')
POSTGRES_DB = os.getenv('POSTGRES_DB')

In [5]:
engine = create_engine(f'postgresql://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}')

In [6]:
Session = sessionmaker(bind=engine)
session = Session()

# Data

In [7]:
sql_query = text("""
WITH global_climate_and_forest_fire AS (
    SELECT * FROM global_climate 
    JOIN forest_fire USING(latitude, longitude, \"date\")
), 
global_climate_and_forest_fire_and_ndvi AS (
    SELECT * FROM global_climate_and_forest_fire 
    JOIN ndvi USING(latitude, longitude, \"date\")
), land_cover_data AS (
    SELECT lc.latitude, lc.longitude, lc."year", 
    lcl."General class", lcl."class", lcl."Sub-class"
    FROM land_cover lc JOIN land_cover_legend lcl
    ON lc.land_cover = lcl."Map value"
),
population_density_and_land_cover AS (
    SELECT * FROM population_density 
    JOIN land_cover_data USING(latitude, longitude, \"year\")
), 
all_data AS (
    SELECT DISTINCT * FROM population_density_and_land_cover pd_lc
    JOIN global_climate_and_forest_fire_and_ndvi gc_ff_nd
    USING(latitude, longitude) 
    WHERE pd_lc.\"year\" = EXTRACT(YEAR FROM gc_ff_nd.\"date\")
)

SELECT * FROM all_data;
""")
data = session.execute(sql_query)

In [8]:
rows = data.fetchall()
df = pd.DataFrame(rows, columns=data.keys())

In [9]:
columns = {
    "latitude": "latitude",
    "longitude": "longitude",
    "population_density": "population_density",
    "General class": "land_cover_type",
    "class": "land_cover_subtype",
    "Sub-class": "vegetation_percent",
    "date": "date",
    "ws": "win_speed",
    "vpd": "vapor_pressure_deficit",
    "vap": "vapor_pressure",
    "tmin": "minimum_temperature",
    "tmax": "maximum_temperature",
    "swe": "snow_water_equivalent",
    "srad": "surface_shortwave_radiation",
    "soil": "soil_moisture",
    "q": "runoff",
    "ppt": "precipitation_accumulation",
    "pet": "Reference_evapotranspiration",
    "def": "climate_water_deficit",
    "aet": "actual_Evapotranspiration",
    "PDSI": "palmer_drought_severity_index",
    "brightness": "brightness_temperature",
    "bright_t31": "brightness_temperature_31",
    "scan": "scan_fire_size",
    "track": "track_fire_size",
    "confidence": "confidence",
    "frp": "fire_radiative_power",
    "daynight": "daynight",
    "type": "fire_type",
    "n_pixels": "n_pixels_ndvi",
    "vim": "ndvi",
    "vim_avg": "ndvi_long_term_average",
    "viq": "ndvi_anomaly_percent"
}

In [10]:
df_final = df.rename(columns=columns)[columns.values()]
df_final.head(5)

,latitude,longitude,population_density,land_cover_type,land_cover_subtype,vegetation_percent,date,win_speed,vapor_pressure_deficit,vapor_pressure,...,scan_fire_size,track_fire_size,confidence,fire_radiative_power,daynight,fire_type,n_pixels_ndvi,ndvi,ndvi_long_term_average,ndvi_anomaly_percent
0,-4.223234,-69.970612,154.694519,Open surface water,Open surface water,90-100% of year,2017-10-14,1.47,0.82,3.060,...,0.38,0.36,l,3.72,D,offshore,198.0,0.8564,0.8548,100.1807
1,-4.220858,-69.962410,199.979401,Built-up,Built-up,Built-up,2020-08-19,1.20,1.00,2.898,...,0.38,0.36,n,3.03,D,offshore,198.0,0.8588,0.8557,100.3475
2,-4.216577,-69.969467,194.736084,Wetland,Dense short vegetation,100% short vegetation cover,2013-09-22,1.15,0.85,2.953,...,0.41,0.37,n,2.84,D,offshore,198.0,0.8528,0.8541,99.8581
3,-4.216006,-69.965797,151.117371,Wetland,Dense short vegetation,100% short vegetation cover,2013-09-22,1.15,0.85,2.953,...,0.41,0.37,n,3.83,D,offshore,198.0,0.8528,0.8541,99.8581
4,-4.213908,-69.962715,147.899872,Wetland,Dense short vegetation,100% short vegetation cover,2016-10-26,1.47,0.89,3.136,...,0.38,0.43,n,3.48,D,offshore,198.0,0.8572,0.8555,100.1846


# Model

(1391746, 1391746)

# Close connection

In [12]:
session.close()
engine.dispose()